In [1]:
import matplotlib.pyplot as plt
import os
import numpy as np
import mne
from antio import read_cnt
from antio.parser import read_triggers
from matplotlib import pyplot as plt
from mne.datasets import testing
from mne.io import read_raw_brainvision
from mne.io import read_raw_ant
from mne.viz import plot_topomap
import glob
from mne.preprocessing import ICA, corrmap, create_ecg_epochs, create_eog_epochs

import scipy.io as scio


from mne.viz import set_browser_backend
set_browser_backend("qt")

#%matplotlib widget

Using qt as 2D backend.


In [2]:
def get_eeg_files_in_folder(path,target):
    files = os.listdir(path)
    target_files = [file for file in files if file.endswith(target)]
    return target_files
def eeg_path(root_path,sub_index):
    participant_list = os.listdir(root_path)
    cnt_path = glob.glob(root_path+participant_list[sub_index]+'/'+'*.cnt')[0]
    evt_path = glob.glob(root_path+participant_list[sub_index]+'/'+'*.evt')[0]
    trg_path = glob.glob(root_path+participant_list[sub_index]+'/'+'*.trg')[0]
    return cnt_path,evt_path,trg_path

def eeg_path_BV(root_path,sub_index):
    participant_list = os.listdir(root_path)
    eeg_path = glob.glob(root_path+participant_list[sub_index]+'/'+'*.eeg')[0]
    vhdr_path = glob.glob(root_path+participant_list[sub_index]+'/'+'*.vhdr')[0]
    vmrk_path = glob.glob(root_path+participant_list[sub_index]+'/'+'*.vmrk')[0]
    return eeg_path,vhdr_path,vmrk_path

def split_session_events(events,event_id):
    button_index = event_id['16']
    sound_index = event_id['96']
    event_list = np.array(events)
    if '64' in event_id:
        event_list[event_list[:,2]==event_id['64'],2]=event_id['96']
    #print(event_list[:,2],'\n',event_id['96'],event_id)
    locker_identifier_1 = [button_index]*2+[sound_index]*3+[button_index]*2+[sound_index]+[button_index]*2+[sound_index]*3+[button_index]*2
    locker_identifier_2 = [button_index]*2+[sound_index]*3+[button_index]*2+[sound_index]+[sound_index]*3+[button_index]*2
    head_trigger_type = [sound_index]*3+[button_index]
    tail_trigger_type = [sound_index]+[button_index]*2+[sound_index]

    locker_head = []
    locker_tail = []
    for i in range(event_list.shape[0]-25):
        if (list(event_list[i:(i+len(locker_identifier_1)+len(head_trigger_type)),2])==locker_identifier_1+head_trigger_type)| \
            (list(event_list[i:(i+len(locker_identifier_2)+len(head_trigger_type)),2])==locker_identifier_2+head_trigger_type):
            locker_head = locker_head + [i+2]
        elif (list(event_list[i:(i+len(locker_identifier_1)+len(tail_trigger_type)),2])==locker_identifier_1+tail_trigger_type)| \
            (list(event_list[i:(i+len(locker_identifier_2)+len(tail_trigger_type)),2])==locker_identifier_2+tail_trigger_type):
            locker_tail = locker_tail + [i+12]
    #print('Head: ',len(locker_head),'.  Tail: ',len(locker_tail))

    arg_sound = np.where(event_list[:,2]==sound_index)[0]

    sound_trigger_index = np.where(events[:,2]==event_id['96'])[0]
    if len(sound_trigger_index)==540:
        head = sound_trigger_index[60]
    else:
        head = locker_head[0]
    #if len(locker_head)>0:
    #    head = locker_head[0]
    #elif len(locker_head)==0:
    #    head = arg_sound[60]

    if len(locker_tail)>0:
        tail = locker_tail[-1]
    elif len(locker_tail)==0:
        tail = arg_sound[np.where(arg_sound==head)[0][0]+420]-1
    train_event = event_list[(head):(tail+1),:]
    
    #print('Head: ',head,'.  Tail: ',tail)
    #plt.scatter(np.arange(len(event_list[(tail-20):(tail+20),2])),event_list[(tail-20):(tail+20),2])
    #plt.scatter(20,event_list[tail,2],c='r')
    #plt.plot(np.arange(len(event_list[(tail-20):(tail+20),2])),event_list[(tail-20):(tail+20),2])

    #print('Head: ',head,'.  Tail: ',tail)
    #plt.scatter(np.arange(len(event_list[(head-20):(head+20),2])),event_list[(head-20):(head+20),2])
    #plt.scatter(20,event_list[head,2],c='r')
    #plt.plot(np.arange(len(event_list[(head-20):(head+20),2])),event_list[(head-20):(head+20),2])
    
    train_sound_event = train_event[train_event[:,2]==sound_index,:]
    
    if train_sound_event.shape[0]!=420:
        raise ValueError("\nIncorrect training session trial number!!!\n")
    else:
        pre_event = event_list[:(head-1),:]
        post_event = event_list[(tail+1):,:]
        pre_sound_event = pre_event[pre_event[:,2]==sound_index,:]
        if pre_sound_event.shape[0]>60:
            pre_sound_event = pre_sound_event[-61:-1,:]
        post_sound_event = post_event[post_event[:,2]==sound_index,:]
        print('Train_sound_event shape: ',train_sound_event.shape,
              ';   Pre_sound_event shape: ',pre_sound_event.shape,
              ';   Post_sound_event shape: ',post_sound_event.shape)
        return train_sound_event,pre_sound_event,post_sound_event

    #return train_sound_event,head,tail

In [3]:
def load_catch_index_for_TRF(file_path):
    file = scio.loadmat(file_path)
    stim_index = file['participant_log']['stim_index'][0][0]
    pre_index = stim_index[stim_index[:,4]==1,3]
    train_index = stim_index[stim_index[:,4]==2,3]
    post_index = stim_index[stim_index[:,4]==3,3]
    pre_post_NS_index = np.zeros(stim_index.shape[0])
    NS_locker = np.where(stim_index[:,1]==1)[0]
    feature_index = []
    for loc in NS_locker:
        pre_post_NS_index[(loc-3):loc] = 1
        pre_post_NS_index[(loc+1):(loc+4)] = 2
        feature_index = feature_index + list(stim_index[(loc-3):loc,0].astype('int16')-1)
        feature_index = feature_index + [int(stim_index[loc,0]-1)]
        feature_index = feature_index + list(stim_index[(loc+1):(loc+4),0].astype('int16')-1)
        
    pre_post_NS_index = pre_post_NS_index[60:60+420]
    
    return pre_index,train_index,post_index,pre_post_NS_index,np.array(feature_index)

### Stop here to check the participant's ID

In [ ]:
if_onset_shifting = True
participant_index = 0

raw_eeg_path = 'Data/'
participant_list = os.listdir(raw_eeg_path)
print(participant_list)
eeg_file_list = os.listdir(raw_eeg_path+participant_list[participant_index]+'/')
#eeg_file_list = glob.glob(raw_eeg_path+participant_list[0]+'/'+'*.cnt')
print(eeg_file_list)

In [ ]:
####################################################
#Try .cnt format
fname,_,_ = eeg_path(raw_eeg_path,participant_index)
#Load impedances
cnt = read_cnt(fname)
_, _, _, impedances, _ = read_triggers(cnt)
#Load Raw Data
raw = read_raw_ant(fname, eog=r"EOG",preload=False)
raw_event = raw.copy()
#set standard 10-20 Montage
#raw_set_montage = raw.copy()
#raw_set_montage.pick("eeg").set_montage("standard_1020")
raw.pick("eeg").set_montage("standard_1020")
#raw.drop_channels(['M1','M2'])


####################################################
#Try .eeg format
#_,fname,_ = eeg_path_BV(raw_eeg_path,0)
#raw = read_raw_brainvision(fname)
#raw.set_channel_types({'EOG': 'eog'})


In [ ]:
raw.load_data()
raw_event.load_data()

#raw = mne.add_reference_channels(raw, ref_channels=["CPz"])
#raw_event = mne.add_reference_channels(raw_event, ref_channels=["CPz"])

#raw.pick("eeg").set_montage("standard_1020")

In [ ]:
#raw.del_proj()
#sphere = mne.make_sphere_model("auto", "auto", raw.info)
#src = mne.setup_volume_source_space(sphere=sphere, exclude=30.0, pos=5.0)
#forward = mne.make_forward_solution(raw.info, trans=None, src=src, bem=sphere)
#raw_rer = raw.copy().set_eeg_reference("REST", forward=forward)

In [ ]:
print(raw.info)
print(raw.info['ch_names'])

In [ ]:
#raw.plot(n_channels=63, remove_dc=False,color='darkblue',scalings=10e-5)

In [ ]:
#raw.info["bads"] = ['T7','T8']

#print(raw.info)
#print(raw.info.ch_names)
#print(raw_event.info)
#print(raw.info["bads"])

In [ ]:
#raw.interpolate_bads(reset_bads=True)

In [ ]:
events,event_id = mne.events_from_annotations(raw)
print(events[:5])
print(events[-5:])
print(len(events))
print(event_id)
print(np.where(events[:,2]==event_id['96'])[0].shape)
#train_sound_event,head,tail = split_session_events(events,event_id)
train_sound_event,pre_event,post_event = split_session_events(events,event_id)

## Add onset shifting

In [ ]:
participant_log_path = 'C:/Users/yu028288/OneDrive - University of Central Florida/Documents/Python Scripts/SWS_OneShotLearning_2025/Experiment/EEG_Experiment/Final_EEG_Behaviour_Result'
log_file_list = sorted(os.listdir(participant_log_path))
_,train_index,_,pre_post_NS_index,feature_index = load_catch_index_for_TRF(participant_log_path+'/'+log_file_list[participant_index])

word_onset_shifting_list = np.loadtxt('word_onset_shifting_list.txt');

#choose True to run onset shifting
if_onset_shifting = True

if if_onset_shifting&(train_sound_event.shape[0]==420):
    word_onset_sample_shifting_list = word_onset_shifting_list[feature_index]*raw.info['sfreq']
    train_sound_event[:,0] = train_sound_event[:,0] + word_onset_sample_shifting_list.astype('int64')

print(train_sound_event[:,:])

### Stop here to check if we have 420 trials

In [ ]:
sound_trigger_index = np.where(events[:,2]==event_id['96'])[0]

In [ ]:
#Crop
crop_tmin = events[sound_trigger_index[0]][0]/1000-0.3
crop_tmax = events[sound_trigger_index[-1]][0]/1000+1.3

print(crop_tmin,'\n',crop_tmax)

raw = raw.crop(tmin=crop_tmin,tmax=crop_tmax)
raw_event = raw_event.crop(tmin=crop_tmin,tmax=crop_tmax)

In [ ]:
raw.plot(n_channels=63, remove_dc=False,color='darkblue',scalings=10e-5)

### Stop here to verify the raw data

In [ ]:
#fig,axis = plt.subplots(1,1,figsize = [100,1])
#axis.plot(events[head:(tail+1),2])
#axis.scatter(np.arange(len(events[head:(tail+1),2])),events[head:(tail+1),2])

In [ ]:
#events_array = np.array(events[head:(tail+1)])
#i_64 = np.where(events_array[:,2]==3)[0][0]
#i_32 = np.where(events_array[:,2]==2)[0][0]
#print(i_64,i_32,events_array[i_64,0],events_array[i_32,0])
#plt.scatter(events_array[(i_64-10):(i_64+10),0],events_array[(i_64-10):(i_64+10),2])

In [ ]:
print(raw.info.ch_names)
#raw.plot(duration=60, n_channels=63, remove_dc=False,color='darkblue',scalings=20e-5)

In [ ]:
fig = raw.compute_psd(tmax=np.inf, fmax=250).plot(
    average=True, amplitude=False, picks="data", exclude="bads"
)

In [ ]:
print(raw.get_montage())

In [ ]:
#Test best band
#for cutoff in (30, 40):
#    raw_lowpass = raw.copy().filter(h_freq=cutoff, l_freq=None,n_jobs=8)
#    raw_lowpass = raw_lowpass.filter(h_freq=None, l_freq=0.3,n_jobs=8)
#    with mne.viz.use_browser_backend("matplotlib"):
#        fig = raw_lowpass.plot(duration=60, n_channels=63, remove_dc=False,color='darkblue',scalings=20e-5)
#    fig.subplots_adjust(top=0.9)
#    fig.suptitle(f"High-pass filtered at {cutoff} Hz", size="xx-large", weight="bold")

# Run ICA

In [ ]:
def ica_include_exclude(or_list,in_list,ex_list):
    for i in ex_list:
        if i not in or_list:
            or_list = or_list + [i]
    for i in in_list:
        if i in or_list:
            or_list.remove(i)
    
    return or_list

def ica_keep_comp(n_ic,keep_list):
    ic_list = list(np.arange(n_ic))
    for i in ic_list:
        if i in keep_list:
            ic_list.remove(i)
    return ic_list

def raw_subtract(raw1,raw2):
    ch_num = raw1[:,:][0].shape[0]
    raw_out = raw1.copy()
    for i in range(ch_num):
        raw_out[i,:][0][0] = raw1[i,:][0][0] - raw2[i,:][0][0]
    return raw_out

In [ ]:
#Fix baseline and pre-filter
raw_remove_baseline = raw.copy()
raw_remove_baseline = raw_remove_baseline.filter(h_freq=None, l_freq=0.1,n_jobs=8)
raw_remove_baseline = raw_remove_baseline.filter(h_freq=45, l_freq=None,n_jobs=8)
raw_eeg_band = raw_remove_baseline.copy().filter(h_freq=15, l_freq=None,n_jobs=8)
raw_eog_band = raw_remove_baseline.copy().filter(h_freq=None, l_freq=1,n_jobs=8)
raw_eog_band = raw_eog_band.filter(h_freq=10, l_freq=None,n_jobs=8)

In [ ]:
#Acquire EOG
eog_evoked = create_eog_epochs(raw_eeg_band,ch_name=['Fp1','Fp2']).average()
eog_evoked.apply_baseline(baseline=(None, -0.2))
eog_evoked.plot_joint()

In [ ]:
ecg_evoked = create_ecg_epochs(raw_eeg_band,ch_name='P8').average()
ecg_evoked.apply_baseline(baseline=(None, -0.2))
ecg_evoked.plot_joint()

In [ ]:
#Start ICA in eog band
ica = ICA(n_components=45, max_iter="auto", random_state=97)
ica.fit(raw_eog_band)
ica

In [ ]:
ica.plot_components()

In [ ]:
eog_indices, eog_scores = ica.find_bads_eog(raw_eog_band,ch_name=['Fp1','AF7','Fp2','AF8','Fpz'],l_freq=0.5, h_freq=10)
ica.plot_scores(eog_scores,exclude=eog_indices)
print(eog_indices)

In [ ]:

ecg_indices = []
#ecg_i, ecg_scores = ica.find_bads_ecg(raw_eeg_band, ch_name='T7',method="correlation", threshold="auto")
#ecg_indices = ecg_indices + ecg_i
#ecg_i, ecg_scores = ica.find_bads_ecg(raw_eeg_band, ch_name='F7',method="correlation", threshold="auto")
#ecg_indices = ecg_indices + ecg_i
#ecg_i, ecg_scores = ica.find_bads_ecg(raw_eeg_band, ch_name='FT7',method="correlation", threshold="auto")
#ecg_indices = ecg_indices + ecg_i
#ecg_i, ecg_scores = ica.find_bads_ecg(raw_eeg_band, ch_name='TP7',method="correlation", threshold="auto")
#ecg_indices = ecg_indices + ecg_i
#ecg_i, ecg_scores = ica.find_bads_ecg(raw_eeg_band, ch_name='P7',method="correlation", threshold="auto")
#ecg_indices = ecg_indices + ecg_i
#ecg_i, ecg_scores = ica.find_bads_ecg(raw_eeg_band, ch_name='T8',method="correlation", threshold="auto")
#ecg_indices = ecg_indices + ecg_i
#ecg_i, ecg_scores = ica.find_bads_ecg(raw_eeg_band, ch_name='F8',method="correlation", threshold="auto")
#ecg_indices = ecg_indices + ecg_i
#ecg_i, ecg_scores = ica.find_bads_ecg(raw_eeg_band, ch_name='FT8',method="correlation", threshold="auto")
#ecg_indices = ecg_indices + ecg_i
#ecg_i, ecg_scores = ica.find_bads_ecg(raw_eeg_band, ch_name='TP8',method="correlation", threshold="auto")
#ecg_indices = ecg_indices + ecg_i
#ecg_i, ecg_scores = ica.find_bads_ecg(raw_eeg_band, ch_name='P8',method="correlation", threshold="auto")
#ecg_indices = ecg_indices + ecg_i
#ecg_i, ecg_scores = ica.find_bads_ecg(raw_eeg_band, ch_name='PO7',method="correlation", threshold="auto")
#ecg_indices = ecg_indices + ecg_i
#ecg_i, ecg_scores = ica.find_bads_ecg(raw_eeg_band, ch_name='PO8',method="correlation", threshold="auto")
#ecg_indices = ecg_indices + ecg_i
ecg_i, ecg_scores = ica.find_bads_ecg(raw_eog_band, ch_name='Oz',method="correlation", threshold="auto")
ecg_indices = ecg_indices + ecg_i
#ecg_i, ecg_scores = ica.find_bads_ecg(raw_eeg_band, ch_name='O1',method="correlation", threshold="auto")
#ecg_indices = ecg_indices + ecg_i
#ecg_i, ecg_scores = ica.find_bads_ecg(raw_eeg_band, ch_name='O2',method="correlation", threshold="auto")
#ecg_indices = ecg_indices + ecg_i
#ecg_indices = list(set(ecg_indices))


In [ ]:
emg_indices, emg_scores = ica.find_bads_muscle(raw_eog_band,threshold=0.65)

In [ ]:
#ica.plot_properties(raw_eeg_band, picks=emg_indices, log_scale=True)

In [ ]:
ica.plot_scores(emg_scores, exclude=emg_indices)

In [ ]:
print(eog_indices,ecg_indices,emg_indices)
ica_exclude = eog_indices+ecg_indices+emg_indices
print(ica_exclude,len(ica_exclude))

In [ ]:
ica_exclude_list = ica_exclude
#ica_exclude_list = np.sort(np.unique(np.array(ica_include_exclude(ica_exclude,[],[44]))))
print(ica_exclude_list,len(ica_exclude_list))

In [ ]:
#ICA denoising result
ica.exclude = list(ica_exclude_list)
#ica.exclude = ica_exclude_list
#ica.exclude = ica_exclude_list + [2]
#ica.exclude = ica_keep_comp(45,eog_indices + [2])
#ica.exclude = eog_indices
print(len(ica.exclude))
ica.plot_overlay(raw_eog_band, exclude=ica.exclude, picks="eeg")

In [ ]:
ica.plot_sources(raw_eog_band,show_scrollbars=True,splash=False)

In [ ]:
reconst_eog = raw_eog_band.copy()

In [ ]:
ica.apply(reconst_eog)
raw_eeg_band.plot(n_channels=63, remove_dc=False,color='darkblue',scalings=50e-6)
reconst_eog.plot(n_channels=63, remove_dc=False,color='darkblue',scalings=50e-6)

In [ ]:
raw_eeg_eogremove = raw_eeg_band.copy()
ica.apply(raw_eeg_eogremove)
#raw_eeg_eogremove = raw_subtract(raw_eeg_band,reconst_eog)
raw_eeg_eogremove.plot(n_channels=63, remove_dc=False,color='darkblue',scalings=50e-6)

### Stop here for ICA quality checking

In [ ]:
reconst_raw = raw_eeg_eogremove

In [ ]:
raw_high_eogremove = raw_remove_baseline.copy()
ica.apply(raw_high_eogremove)

In [ ]:
if_detrend = 1

# Low Band

### ICA

In [ ]:
ica = ICA(n_components=45, max_iter="auto", random_state=97)
ica.fit(reconst_raw)

ecg_indices = []
ecg_i, ecg_scores = ica.find_bads_ecg(reconst_raw, ch_name='Oz',method="correlation", threshold="auto")
ecg_indices = ecg_indices + ecg_i
print('\n',ecg_indices,'\n','\n')

ica.exclude = ecg_indices
print(len(ica.exclude))
ica.plot_overlay(reconst_raw, exclude=ica.exclude, picks="eeg")

ica.apply(reconst_raw)

## Run Filter

In [ ]:
raw_bandpass = reconst_raw.copy().filter(h_freq=15, l_freq=None,n_jobs=8)
raw_bandpass = raw_bandpass.filter(h_freq=None, l_freq=0.5,n_jobs=8)
fig = reconst_raw.compute_psd(tmax=np.inf, fmax=250).plot(
    average=True, amplitude=False, picks="data", exclude="bads"
)
fig = raw_bandpass.compute_psd(tmax=np.inf, fmax=250).plot(
    average=True, amplitude=False, picks="data", exclude="bads"
)

In [ ]:
raw.plot(n_channels=63, remove_dc=False,color='darkblue',scalings=1e-5)
reconst_raw.plot(n_channels=63, remove_dc=False,color='darkblue',scalings=1e-5)
raw_bandpass.plot(n_channels=63, remove_dc=False,color='darkblue',scalings=1e-5)

## Split Epochs

In [ ]:
def split_train_triggers(train_sound_event):
    if train_sound_event.shape[0]!=420:
        raise ValueError("\nIncorrect training session trial number!!!\n")
    else:
        train_triggers = []
        small_block = [1,1,1,3,2,2,2]
        for i in range(60):
            train_triggers = train_triggers + small_block
        event_id = {'SWS_Pre': 5, 'SWS_Post': 6, 'NS': 7}
        train_triggers = np.array(train_triggers)+4
        train_triggers = np.vstack([train_sound_event[:,:2].T,train_triggers])
        return train_triggers.T,event_id
        
        

In [ ]:
train_triggers,train_event_id = split_train_triggers(train_sound_event)

In [ ]:
erp_baseline = (-0.2,0)

epochs_train = mne.Epochs(raw_bandpass, list(train_triggers),event_id = train_event_id, tmin=-0.2, tmax=1, baseline = erp_baseline, preload=False, detrend=if_detrend)
#epochs_train = epochs_train.apply_baseline()
epochs_pre = mne.Epochs(raw_bandpass, list(pre_event), tmin=-0.2, tmax=1, baseline = erp_baseline, preload=False, detrend=if_detrend)
#epochs_pre = epochs_pre.apply_baseline()
epochs_post = mne.Epochs(raw_bandpass, list(post_event), tmin=-0.2, tmax=1, baseline = erp_baseline, preload=False, detrend=if_detrend)
#epochs_post = epochs_post.apply_baseline()

epochs = mne.Epochs(raw_bandpass, list(np.vstack([pre_event,train_sound_event,post_event])), tmin=-0.2, tmax=0.8, baseline = erp_baseline, preload=False, detrend=if_detrend)
epochs = epochs.apply_baseline()

print(epochs.info.ch_names)

In [ ]:
epochs_train.plot(n_epochs=20, events=train_sound_event,n_channels=63,scalings=2e-5)

In [ ]:
spectrum = epochs_train.compute_psd()
spectrum.plot_topomap()

In [ ]:
evoked = epochs_train.average()
evoked.plot(spatial_colors=True)

In [ ]:
evoked.plot_joint(times='peaks')

In [ ]:
evoked.plot_image()

In [ ]:
mne.viz.plot_compare_evokeds(
    evoked,
    axes="topo",
)

In [ ]:
epochs_train.compute_psd().plot(picks="eeg", exclude="bads", amplitude=False)

## Export epochs

In [ ]:
print(participant_index)
export_fname = raw_eeg_path+participant_list[participant_index]+'/' + participant_list[participant_index] + '_blk-2_lowband_onsetshifting-epo.fif'
epochs_train.save(export_fname,overwrite=True)
export_fname = raw_eeg_path+participant_list[participant_index]+'/' + participant_list[participant_index] + '_blk-1_lowband_onsetshifting-epo.fif'
epochs_pre.save(export_fname,overwrite=True)
export_fname = raw_eeg_path+participant_list[participant_index]+'/' + participant_list[participant_index] + '_blk-3_lowband_onsetshifting-epo.fif'
epochs_post.save(export_fname,overwrite=True)

In [ ]:
epochs_blk2 = mne.read_epochs(raw_eeg_path+participant_list[participant_index]+'/' + participant_list[participant_index] + '_blk-2_lowband_onsetshifting-epo.fif', proj=True, preload=False)

spectrum = epochs_blk2.compute_psd()
spectrum.plot_topomap()

In [ ]:
evoked = epochs_blk2.average()
evoked.plot(spatial_colors=True)

# High Band

### ICA

In [ ]:
ica = ICA(n_components=45, max_iter="auto", random_state=97)
ica.fit(raw_high_eogremove)

ecg_indices = []
ecg_i, ecg_scores = ica.find_bads_ecg(raw_high_eogremove, ch_name='Oz',method="correlation", threshold="auto")
ecg_indices = ecg_indices + ecg_i
print('\n',ecg_indices,'\n','\n')
emg_indices = []

### ICA remove EMG

In [ ]:
emg_indices, emg_scores = ica.find_bads_muscle(raw_eog_band,threshold=0.65)
ica.plot_scores(emg_scores, exclude=emg_indices)

In [ ]:
ica.exclude = ecg_indices + emg_indices
print(len(ica.exclude))
ica.plot_overlay(raw_high_eogremove, exclude=ica.exclude, picks="eeg")

ica.apply(raw_high_eogremove)

## Run Filter

In [ ]:
raw_bandpass = raw_high_eogremove.copy().filter(h_freq=45, l_freq=None,n_jobs=8)
raw_bandpass = raw_bandpass.filter(h_freq=None, l_freq=0.1,n_jobs=8)
fig = reconst_raw.compute_psd(tmax=np.inf, fmax=250).plot(
    average=True, amplitude=False, picks="data", exclude="bads"
)
fig = raw_bandpass.compute_psd(tmax=np.inf, fmax=250).plot(
    average=True, amplitude=False, picks="data", exclude="bads"
)

In [ ]:
raw.plot(n_channels=63, remove_dc=False,color='darkblue',scalings=1e-5)
reconst_raw.plot(n_channels=63, remove_dc=False,color='darkblue',scalings=1e-5)
raw_bandpass.plot(n_channels=63, remove_dc=False,color='darkblue',scalings=1e-5)

## Split Epochs

In [ ]:
def split_train_triggers(train_sound_event):
    if train_sound_event.shape[0]!=420:
        raise ValueError("\nIncorrect training session trial number!!!\n")
    else:
        train_triggers = []
        small_block = [1,1,1,3,2,2,2]
        for i in range(60):
            train_triggers = train_triggers + small_block
        event_id = {'SWS_Pre': 5, 'SWS_Post': 6, 'NS': 7}
        train_triggers = np.array(train_triggers)+4
        train_triggers = np.vstack([train_sound_event[:,:2].T,train_triggers])
        return train_triggers.T,event_id
        
        

In [ ]:
train_triggers,train_event_id = split_train_triggers(train_sound_event)

In [ ]:
erp_baseline = (-0.2,0)

epochs_train = mne.Epochs(raw_bandpass, list(train_triggers),event_id = train_event_id, tmin=-0.2, tmax=1, baseline = erp_baseline, preload=False, detrend=if_detrend)
#epochs_train = epochs_train.apply_baseline()
epochs_pre = mne.Epochs(raw_bandpass, list(pre_event), tmin=-0.2, tmax=1, baseline = erp_baseline, preload=False, detrend=if_detrend)
#epochs_pre = epochs_pre.apply_baseline()
epochs_post = mne.Epochs(raw_bandpass, list(post_event), tmin=-0.2, tmax=1, baseline = erp_baseline, preload=False, detrend=if_detrend)
#epochs_post = epochs_post.apply_baseline()

epochs = mne.Epochs(raw_bandpass, list(np.vstack([pre_event,train_sound_event,post_event])), tmin=-0.2, tmax=0.8, baseline = erp_baseline, preload=False, detrend=if_detrend)
epochs = epochs.apply_baseline()

print(epochs.info.ch_names)

In [ ]:
epochs_train.plot(n_epochs=20, events=train_sound_event,n_channels=63,scalings=2e-5)

In [ ]:
spectrum = epochs_train.compute_psd()
spectrum.plot_topomap()

In [ ]:
evoked = epochs_train.average()
evoked.plot(spatial_colors=True)

In [ ]:
evoked.plot_joint(times='peaks')

In [ ]:
evoked.plot_image()

In [ ]:
mne.viz.plot_compare_evokeds(
    evoked,
    axes="topo",
)

In [ ]:
epochs_train.compute_psd().plot(picks="eeg", exclude="bads", amplitude=False)

## Export epochs

In [ ]:
print(participant_index)
export_fname = raw_eeg_path+participant_list[participant_index]+'/' + participant_list[participant_index] + '_blk-2_highband_onsetshifting-epo.fif'
epochs_train.save(export_fname,overwrite=True)
export_fname = raw_eeg_path+participant_list[participant_index]+'/' + participant_list[participant_index] + '_blk-1_highband_onsetshifting-epo.fif'
epochs_pre.save(export_fname,overwrite=True)
export_fname = raw_eeg_path+participant_list[participant_index]+'/' + participant_list[participant_index] + '_blk-3_highband_onsetshifting-epo.fif'
epochs_post.save(export_fname,overwrite=True)

In [ ]:
epochs_blk2 = mne.read_epochs(raw_eeg_path+participant_list[participant_index]+'/' + participant_list[participant_index] + '_blk-2_highband_onsetshifting-epo.fif', proj=True, preload=False)

spectrum = epochs_blk2.compute_psd()
spectrum.plot_topomap()